# Ch10-3.RNN: Sequence2Sequence

### Seq2Seq 번역하기(구글 번역)

#### (예) word -> 번역: 단어

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
#소문자 a~z까지 배열변수에 넣기
#S:decoder Start,  E: decoder End, P:심볼
char_arr=[c for c in 'SEPabcdefghijklmnopqrstuvwxyz단어나무놀이소녀키스사랑']
num_dic={n:i for i,n in enumerate(char_arr)}
dic_len=len(num_dic)

In [3]:
seq_data = [['word', '단어'], ['wood', '나무'],
            ['game', '놀이'], ['girl', '소녀'],
            ['kiss', '키스'], ['love', '사랑']]

In [4]:
def make_batch(seq_data):
    input_batch=[]
    output_batch=[]
    target_batch=[]
    
    for seq in seq_data:
        input=[num_dic[n] for n in seq[0]]
        output=[num_dic[n] for n in ('S'+seq[1])]
        target=[num_dic[n] for n in (seq[1]+'E')]
        
        input_batch.append(np.eye(dic_len)[input])
        output_batch.append(np.eye(dic_len)[output])
        target_batch.append(target)
        
    return input_batch, output_batch, target_batch

In [5]:
learning_rate=0.01 #최적화함수에서 사용할 학습률
total_epoch=100#전체 데이터를 학습할 총 횟수
n_hidden=128 #hidden layer의 뉴런 갯수
n_input=n_class=dic_len

In [6]:
enc_input=tf.placeholder(tf.float32,[None,None, n_input]) 
dec_input=tf.placeholder(tf.float32,[None,None,n_input]) 
targets=tf.placeholder(tf.int64,[None,None])

In [7]:
with tf.variable_scope('encode'):
    enc_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
    enc_cell = tf.nn.rnn_cell.DropoutWrapper(enc_cell, output_keep_prob=0.5)

    outputs, enc_states = tf.nn.dynamic_rnn(enc_cell, enc_input, dtype=tf.float32)

Instructions for updating:
This class is equivalent as tf.keras.layers.SimpleRNNCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [8]:
with tf.variable_scope('decode'):
    dec_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
    dec_cell = tf.nn.rnn_cell.DropoutWrapper(dec_cell, output_keep_prob=0.5)

    outputs, dec_states = tf.nn.dynamic_rnn(dec_cell, dec_input, initial_state=enc_states, dtype=tf.float32)

In [9]:
model = tf.layers.dense(outputs, n_class, activation=None)

cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=model, labels=targets))

optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.


In [10]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

input_batch, output_batch, target_batch = make_batch(seq_data)

for epoch in range(total_epoch):

    _, loss = sess.run([optimizer, cost], feed_dict={enc_input: input_batch,
                                                     dec_input: output_batch,
                                                     targets: target_batch})

    print('Epoch: ', '%04d' % (epoch + 1),
          'Cost= ', '{:.6f}'.format(loss))

print('최적화 완료!')


Epoch:  0001 Cost=  3.755042
Epoch:  0002 Cost=  2.514495
Epoch:  0003 Cost=  1.681527
Epoch:  0004 Cost=  1.132293
Epoch:  0005 Cost=  0.635849
Epoch:  0006 Cost=  0.470242
Epoch:  0007 Cost=  0.257953
Epoch:  0008 Cost=  0.271767
Epoch:  0009 Cost=  0.202466
Epoch:  0010 Cost=  0.233496
Epoch:  0011 Cost=  0.070391
Epoch:  0012 Cost=  0.084783
Epoch:  0013 Cost=  0.079297
Epoch:  0014 Cost=  0.032873
Epoch:  0015 Cost=  0.037073
Epoch:  0016 Cost=  0.101700
Epoch:  0017 Cost=  0.021992
Epoch:  0018 Cost=  0.024138
Epoch:  0019 Cost=  0.071132
Epoch:  0020 Cost=  0.020519
Epoch:  0021 Cost=  0.005445
Epoch:  0022 Cost=  0.009812
Epoch:  0023 Cost=  0.006248
Epoch:  0024 Cost=  0.012702
Epoch:  0025 Cost=  0.002832
Epoch:  0026 Cost=  0.002740
Epoch:  0027 Cost=  0.004442
Epoch:  0028 Cost=  0.002861
Epoch:  0029 Cost=  0.002760
Epoch:  0030 Cost=  0.006013
Epoch:  0031 Cost=  0.004151
Epoch:  0032 Cost=  0.004477
Epoch:  0033 Cost=  0.004534
Epoch:  0034 Cost=  0.005781
Epoch:  0035 C

In [11]:
def translate(word):

    seq_data = [word, 'P' * len(word)]

    input_batch, output_batch, target_batch = make_batch([seq_data])

    # 1argmax(model, 2): 3차원
    prediction = tf.argmax(model, 2)

    result = sess.run(prediction, feed_dict={enc_input: input_batch,
                                             dec_input: output_batch,
                                             targets: target_batch})

    # 10.3.10
    decoded = [char_arr[i] for i in result[0]]

    end = decoded.index('E')
    translated = ''.join(decoded[:end])

    return translated

# 10.3.11
print('\n=== 번역 테스트 ===')

print('word ->', translate('word'))
print('wodr ->', translate('wodr')) ##### 알수없는 값
print('love ->', translate('love'))
print('loev ->', translate('loev'))
print('abcd ->', translate('abcd')) #### 알수없는 값


=== 번역 테스트 ===
word -> 단어
wodr -> 나무
love -> 사랑
loev -> 사랑
abcd -> 키스
